In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/ComputerVision/Covid19-Detector/

Mounted at /content/gdrive
/content/gdrive/MyDrive/ComputerVision/Covid19-Detector


In [2]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=e5c22b65a2ce303a910ab57142fed53c3c2535228732c2ba50f873ceefc3615d
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [3]:
from data_loader import DatasetLoader
from simplepreprocessor import SimplePreprocessor
from imagetoarraypreprocessor import ImageToArrayPreprocessor
from neural_network_architecture import  LeNet, MiniVGGNet, ShallowNet

importing Jupyter notebook from data_loader.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
importing Jupyter notebook from simplepreprocessor.ipynb
importing Jupyter notebook from imagetoarraypreprocessor.ipynb
importing Jupyter notebook from neural_network_architecture.ipynb


In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import numpy as np

In [5]:

WIDTH = 224
HEIGHT = 224

sp = SimplePreprocessor(WIDTH, HEIGHT)
iap = ImageToArrayPreprocessor()

In [6]:
dl = DatasetLoader(preprocessors=[sp, iap])

In [7]:
(data, labels) = dl.load_data()

In [8]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=412)

In [9]:
# convert the labels from integers to vectors
#trainY = LabelBinarizer().fit_transform(trainY)
#testY = LabelBinarizer().fit_transform(testY)

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le = LabelEncoder().fit(trainY)
trainY = to_categorical(le.transform(trainY), 2)
testY = to_categorical(le.transform(testY), 2)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# initialize an our data augmenter 
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

In [11]:
print("Compiling MiniVGGNet  Model")


INIT_LR = 1e-3
BS = 32
EPOCHS = 100
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / EPOCHS)
model = LeNet.build(width=WIDTH, height=HEIGHT, depth=3, classes=2)
model.compile(loss= "binary_crossentropy", optimizer=opt,
metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)


Compiling MiniVGGNet  Model


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


[INFO] training network...
Epoch 1/100
124/124 [==============================] - 48s 312ms/step - loss: 0.1322 - accuracy: 0.9773 - val_loss: 0.0826 - val_accuracy: 0.9842
Epoch 2/100
124/124 [==============================] - 37s 301ms/step - loss: 0.0982 - accuracy: 0.9768 - val_loss: 0.0435 - val_accuracy: 0.9842
Epoch 3/100
124/124 [==============================] - 38s 303ms/step - loss: 0.0660 - accuracy: 0.9801 - val_loss: 0.0343 - val_accuracy: 0.9857
Epoch 4/100
124/124 [==============================] - 38s 303ms/step - loss: 0.0760 - accuracy: 0.9783 - val_loss: 0.0318 - val_accuracy: 0.9865
Epoch 5/100
124/124 [==============================] - 38s 302ms/step - loss: 0.0638 - accuracy: 0.9786 - val_loss: 0.0241 - val_accuracy: 0.9887
Epoch 6/100
124/124 [==============================] - 38s 303ms/step - loss: 0.0585 - accuracy: 0.9808 - val_loss: 0.0258 - val_accuracy: 0.9887
Epoch 7/100
124/124 [==============================] - 37s 301ms/step - loss: 0.0562 - accuracy: 

In [12]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1), target_names=le.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    Negative       0.99      1.00      1.00      1312
    positive       0.76      0.62      0.68        21

    accuracy                           0.99      1333
   macro avg       0.88      0.81      0.84      1333
weighted avg       0.99      0.99      0.99      1333

